In [1]:
import mne
import os
from ieeg.viz.mri import plot_subj, subject_to_info, plot_on_average
from ieeg.io import raw_from_layout, get_data
from misc_functions import load_sig_chans, channel_names_to_indices

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### plot single subject's significant electrodes

In [2]:
subj = "D0071"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'
stim_sig_chans = load_sig_chans(stim_filename)
# resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response_fixationCrossBase_1sec_mirror.json'
# resp_sig_chans = load_sig_chans(stim_filename)


subj_without_zeros = "D" + subj[1:].lstrip('0')
plot_subj(subj_without_zeros, picks=stim_sig_chans, labels_every=1, hemi='both', title=f'{subj}_Stimulus', background='gray', color=(1, 1, 0)) 
# plot_subj(subj_without_zeros, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', color=(1, 1, 0))



Loaded significant channels for subject D0071


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Using pyvistaqt 3d backend.

Could not estimate rigid Talairach alignment, using identity matrix


In [ ]:
subj = "D0063"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'
congruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_congruent.json'
incongruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_incongruent.json'
repeat_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_repeat.json'
switch_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_switch.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)
congruent_sig_chans = load_sig_chans(congruent_filename)
incongruent_sig_chans = load_sig_chans(incongruent_filename)
repeat_sig_chans = load_sig_chans(repeat_filename)
switch_sig_chans = load_sig_chans(switch_filename)


subj_without_zeros = "D" + subj[1:].lstrip('0')
plot_subj(subj_without_zeros, picks=stim_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Stimulus', color=(1, 1, 0)) 
plot_subj(subj_without_zeros, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', color=(1, 1, 0))
plot_subj(subj_without_zeros, picks=congruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Congruent', color=(1, 1, 0)) 
plot_subj(subj_without_zeros, picks=incongruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Incongruent', color=(1, 1, 0))
plot_subj(subj_without_zeros, picks=repeat_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Repeat', color=(1, 1, 0)) 
plot_subj(subj_without_zeros, picks=switch_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Switch', color=(1, 1, 0))


### plot all significant electrodes on average brain

In [15]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    if subj == "D0077":
        continue
    
    resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_response_experimentStartBase1secTo101Sec.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_stimulus_experimentStartBase1secTo101Sec.json'

    resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    combined_chans = resp_sig_chans + stim_sig_chans

    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes

picks = [f'D{k[1:].zfill(4)}-{v}' for k in allSigElectrodes.keys() for v in allSigElectrodes[k]]
# sinfos = [subject_to_info(f'D{int(subj[1:])}') for subj in subjects]

plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))
# plot_on_average(allElectrodes, hemi='both', color=(1,1,0))

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 8 | Sessions: 0 | Runs: 32}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 8 | Sessions: 0 | Runs: 32})
Loaded significant channels for subject D0057
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0071


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jz421\\Box\\CoganLab\\BIDS-1.1_GlobalLocal\\BIDS\\derivatives\\freqFilt\\figs\\D0090\\sig_chans_D0090_response_experimentStartBase1secTo101Sec.json'

### plot all significant electrodes, then plot all electrodes

In [18]:
import mne
print(mne.__version__)

1.6.1


In [20]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    if subj == "D0077":
        continue
    
    # resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'

    # resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    combined_chans = resp_sig_chans + stim_sig_chans
    combined_chans = stim_sig_chans # delete this line once we have responses for everyone
    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes


plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))
# plot_on_average(allElectrodes, hemi='both', color=(1,1,0))

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 8 | Sessions: 0 | Runs: 32}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 8 | Sessions: 0 | Runs: 32})
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0090


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:678: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

OSError: [WinError 1006] The volume for a file has been externally altered so that the opened file is no longer valid: 'C:\\Users\\jz421\\Box\\ECoG_Recon\\D69\\mri\\transforms\\talairach.xfm'

### plot all stim electrodes for an roi
get the subject: electrode mappings from roi_analysis first

In [3]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# List of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090']

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090


In [6]:
import json

# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

Loaded data from subjects_electrodestoROIs_dict.json


In [7]:
def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
    """
    Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

    Args:
    subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
    sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
    roi_list (list): A list of ROIs to filter electrodes.

    Returns:
    dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
    """
    filtered_electrodes_per_subject = {}

    for sub, electrodes_dict in subjects_electrodes_dict.items():
        filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
                    if any(roi in key for roi in roi_list)}

        # Aggregate electrodes into a list for each subject
        filtered_electrodes = []
        for electrodes in filtered.values():
            filtered_electrodes.extend(electrodes)

        filtered_electrodes_per_subject[sub] = filtered_electrodes
        print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

    # Now filter for significant electrodes
    sig_filtered_electrodes_per_subject = {}

    for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
        # Retrieve the list of significant channels for the subject
        sig_chans = sig_chans_per_subject.get(sub, [])

        # Find the intersection of filtered electrodes and significant channels for the subject
        sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

        # Store the significant filtered electrodes for the subject
        sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
        print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

    return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# Example usage:
dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)
dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
acc_electrodes_per_subject, sig_acc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

For subject D0057, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12']
For subject D0059, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LASF10', 'LASF14', 'LASF15', 'LASF16', 'LMSF5', 'LMSF6', 'LMSF12', 'LPSF10', 'LPSF12', 'RAI10', 'RAI11', 'RAI16', 'RAMF11', 'RAMF12', 'RAMF13', 'RMMF13', 'RMMF14', 'RASF15', 'RMSF8', 'RMSF9', 'RMSF10', 'RMSF7', 'RAMF8', 'RAMF9', 'RAMF10', 'RMMF9', 'RMMF10']
For subject D0065, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RASF13', 'RASF14', 'RASF15', 'RMSF11', 'RMSF12', 'RMSF13', 'RMSF14']
For subject D0069, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: []
For subject D0071, G_fron

In [ ]:
# grab all of the significant electrodes in a roi, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:

    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'

    stim_sig_chans = load_sig_chans(stim_filename)


    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes


plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))
# plot_on_average(allElectrodes, hemi='both', color=(1,1,0))

### plot each subject one by one on the fsaverage brain

In [3]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]
allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

subj = 'D0071'
    
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)

# Combine the significant electrodes for this subject
combined_chans = resp_sig_chans + stim_sig_chans

# Add them to the dictionary, tagged with the subject without zeroes

subj_without_zeroes = "D" + subj[1:].lstrip('0')
allSigElectrodes[subj_without_zeroes] = combined_chans

# Get all electrodes for this subject and add them to the allElectrodes dict
info = subject_to_info(subj_without_zeroes)
allSubjElectrodes = mne.pick_channels(info.ch_names, [])
allElectrodes[subj_without_zeroes] = allSubjElectrodes


plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))
plot_on_average(allElectrodes, hemi='both', color=(1,1,0))

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24})
Loaded significant channels for subject D0071
Loaded significant channels for subject D0071


c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:580: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\viz\mri.py:580: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


fig: <mne.viz._brain._brain.Brain object at 0x0000012F0545D4D0>
info: <Info | 8 non-empty values
 bads: []
 ch_names: RFO16, RFO14, RFO13, RFO12, RFO4, RFO3, RFO2, RIA16, RIA15, ...
 chs: 127 sEEG
 custom_ref_applied: False
 dig: 165 items (3 Cardinal, 162 EEG)
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 127
 projs: []
 sfreq: 2000.0 Hz
>
sub: D71
pos: OrderedDict([('RFO16', array([ 0.06118873,  0.0406194 , -0.00212834])), ('RFO14', array([ 0.05369581,  0.03893661, -0.00509436])), ('RFO13', array([ 0.05002104,  0.03836162, -0.00650167])), ('RFO12', array([ 0.04634628,  0.03778664, -0.00790898])), ('RFO4', array([ 0.01623097,  0.0305224 , -0.0199245 ])), ('RFO3', array([ 0.01248445,  0.02968095, -0.02140752])), ('RFO2', array([ 0.00873805,  0.0288396 , -0.02289051])), ('RIA16', array([0.02596348, 0.03897971, 0.05320638])), ('RIA15', array([0.02672262, 0.03660197, 0.04913819])), ('RIA14', array([0.02748176, 0.03422424, 0.04507001])), ('RIA13', array([0.02810575,

IndexError: Column index is out of range (1)